In [1]:
from importlib import reload
def reload_all():
    import tsm; reload(tsm)
    import _keys; reload(_keys)
    import blizzard; reload(blizzard)
    import config; reload(config)
    import kvstore; reload(kvstore)
    import snapshot; reload(snapshot)
    import elemental_arbitrage; reload(elemental_arbitrage)
    import bliz_tsm_join; reload(bliz_tsm_join)
    import crafting; reload(crafting)
    import kvstore; reload(kvstore)
    import crafting; reload(crafting)
reload_all()
from itables import show

In [2]:
from snapshot import SnapshotProcessor
from tsm import auction_house_snapshot
from config import tsm_region_id
from config import tsm_realm_id
from config import tsm_ah_id
from config import tsm_cache_dir

def tsm_ah_snapper():
    return auction_house_snapshot(tsm_region_id, tsm_realm_id, tsm_ah_id)

tsm_ah_snap = SnapshotProcessor(tsm_ah_snapper, cache_dir=tsm_cache_dir)
tsm_ah = tsm_ah_snap.get(max_age_seconds=3000)

In [3]:
from snapshot import SnapshotProcessor
from blizzard import auction_data
from blizzard import auction_summary
from config import blizzard_realm_id
from config import blizzard_ah_id
from config import blizzard_cache_dir

def bliz_ah_snapper():
    return auction_data(blizzard_realm_id, blizzard_ah_id)

bliz_ah_snap = SnapshotProcessor(bliz_ah_snapper, cache_dir=blizzard_cache_dir)
bliz_ah = bliz_ah_snap.get(max_age_seconds=3000)

In [4]:
from kvstore import InefficientKVStore
from blizzard import ItemLookup
from config import blizzard_item_cache
from config import blizzard_item_reverse_cache

items = ItemLookup(
    InefficientKVStore(blizzard_item_cache),
    InefficientKVStore(blizzard_item_reverse_cache),
)

In [5]:
from bliz_tsm_join import item_info_getter
item_info = item_info_getter(items, bliz_ah_snap, tsm_ah_snap, max_age_seconds=3000)
item_info_current = item_info_getter(items, bliz_ah_snap, tsm_ah_snap, max_age_seconds=0)

In [6]:
reload_all()

from crafting import Recipes
from crafting import purchase_price
from crafting import crafting_price
from crafting import gold


r = Recipes(items)
i = r.ingredients

def buy(item, price_method=gold("realm_market_value")):
    return purchase_price(item_info, item, price_method)


recipe_listing = """
2 flask of the frost wyrm
5 icethorn + 5 lichbloom + frost lotus + enchanted vial

2 flask of endless rage
7 lichbloom + 3 goldclover + frost lotus + enchanted vial

2 flask of stoneblood
7 lichbloom + 3 crystallized life + frost lotus + enchanted vial

firecracker salmon
glacial salmon + northern spices

elixir of greater agility
sungrass + goldthorn + crystal vial

titanium bar
8 saronite bar

titanium bar
2 titanium ore

titansteel bar
3 titanium bar + eternal fire + eternal earth + eternal shadow

indestructible potion
2 icethorn + imbued vial

saronite bar
2 saronite ore

potion of speed
2 adder's tongue + pygmy oil + imbued vial

potion of wild magic
2 lichbloom + pygmy oil + imbued vial

titansteel spellblade
6 titansteel bar + 6 infinite dust + 2 frozen orb

glyph of wild growth
resilient parchment + ethereal ink

spellweave robe
8 spellweave + 6 bolt of imbued frostweave + eternium thread + frozen orb
"""

reversible_recipes = """
eternal shadow
10 crystallized shadow

eternal water
10 crystallized water

greater planar essence
3 lesser planar essence

greater cosmic essence
3 lesser cosmic essence
"""

r_out = None
for line in recipe_listing.splitlines():
    if line.strip():
        if r_out:
            r.recipe_from_strings(r_out, line.strip())
            r_out = None
        else:
            r_out = line.strip()

            
r_out = None
for line in reversible_recipes.splitlines():
    if line.strip():
        if r_out:
            r.recipe_from_strings(r_out, line.strip())
            r.recipe_from_strings(line.strip(), r_out)
            r_out = None
        else:
            r_out = line.strip()

In [7]:
buy(i("hi-explosive bomb"))

0.1846

In [8]:
# to journeyman
buy(
    ing_to_journeyman := i("60 rough stone + 30 copper bar + 6 copper bar"),
    #gold("min"),
    gold("realm_market_value"),
)

27.2196

In [9]:
# to expert
buy(
    ing_to_expert := i("60 coarse stone + 20 linen cloth + 5 silver bar + 50 bronze bar + 25 weak flux + 10 moss agate"),
    #gold("min"),
    gold("realm_market_value"),
)

262.51099999999997

In [10]:
# to artisan
buy(
    ing_to_artisan := i("30 heavy stone + 30 bronze bar + 15 wool cloth + 30 bronze bar + 15 medium leather + 15 wool cloth + 30 wool cloth + 4 steel bar + 120 solid stone + 21 mithril bar"),
    #gold("min"),
    gold("realm_market_value"),
)

242.09159999999997

In [11]:
# to master
buy(
    ing_to_master := i("20 mithril bar + 20 mageweave cloth + 120 mithril bar + 60 dense stone + 105 thorium bar + 35 runecloth + 30 thorium bar"),
    #gold("min"),
    gold("realm_market_value"),
)

781.4785

In [12]:
# to wotlk
buy(
    ing_to_wotlk := i("40 fel iron bar + 20 mote of fire + 40 mote of earth + 30 fel iron bar + 40 adamantite bar + 60 netherweave cloth"),
    #gold("min"),
    gold("realm_market_value"),
)

543.2149999999999

In [13]:
# to 375
buy(
    ing_to_375 := i("70 cobalt bar + 42 cobalt bar + 14 crystallized water"),
    #gold("min"),
    gold("realm_market_value"),
)

238.8148

In [14]:
from crafting import crafting_price
from crafting import gold

def approaches(item_name):
    market_craft = crafting_price(
        item_info,
        r.lookup(item_name=item_name)[0],
        in_price_method=gold("realm_market_value"),
        out_price_method=gold("realm_market_value"),
    )
    flip_craft = crafting_price(
        item_info,
        r.lookup(item_name=item_name)[0],
        in_price_method=gold("min"),
        out_price_method=gold("min"),
    )
    hold_craft = crafting_price(
        item_info,
        r.lookup(item_name=item_name)[0],
        in_price_method=gold("min"),
        out_price_method=gold("realm_market_value"),
    )
    return {"market": market_craft, "flip": flip_craft, "hold": hold_craft}

In [15]:
from elemental_arbitrage import print_elemental_arbitrage_report

print_elemental_arbitrage_report(tsm_ah, items)

SKIP        eternal-life
SKIP        eternal-earth
SKIP        eternal-air
DEFINITELY   BS eternal-fire (unit profit: 44095,headroom: 1328)
DEFINITELY   BS eternal-water (unit profit: 27782,headroom: 2360)
DEFINITELY   BS eternal-shadow (unit profit: 48364,headroom: 2301)
YES          SB primal-life (unit profit: 57640,headroom: 295)
SKIP        primal-earth
SKIP        primal-air
SKIP        primal-fire
SKIP        primal-water
SKIP        primal-shadow
DEFINITELY   SB essence-eternal (unit profit: 16085,headroom: 1173)
DEFINITELY   BS essence-cosmic (unit profit: 70228,headroom: 750)
SKIP        essence-nether
SKIP        essence-planar
SKIP        essence-astral
SKIP        essence-magic
SKIP        essence-mystic


In [16]:
from cytoolz import groupby
import pandas as pd

df_sales = pd.read_csv("Accounting_Earthfury_sales.csv")
df_sales["mode"] = "sold"
df_purchases = pd.read_csv("Accounting_Earthfury_purchases.csv")
df_purchases["mode"] = "purchased"
df_expired = pd.read_csv("Accounting_Earthfury_expired.csv")
df_expired["mode"] = "expired"
df_canceled = pd.read_csv("Accounting_Earthfury_canceled.csv")
df_canceled["mode"] = "canceled"
df_all = pd.concat([df_sales, df_purchases, df_expired, df_canceled])

entries = list(df_all.transpose().to_dict().values())
grouped = groupby(lambda entry: entry["itemString"], entries)
names = {entry["itemString"]: entry["itemName"] for entry in entries}
nets = {
    k: sum(
        (
            vv["price"]*vv["quantity"] if vv["mode"] == "sold" else
            -vv["price"]*vv["quantity"] if vv["mode"] == "purchased" else
            0
        ) for vv in v
    )
    for (k, v) in grouped.items()
}
ah_events = {k: [vv["mode"] for vv in v if vv["source"] != "Vendor"] for (k, v) in grouped.items()}
sale_rate = {
    k: v.count("sold") / (v.count("sold") + v.count("expired") + v.count("canceled"))
    for (k, v) in ah_events.items()
    if v.count("sold") > 1
}
agg = {k: {"name": names[k], "net": nets[k], "rate": sale_rate[k], "num": len(ah_events[k])} for k in sale_rate}
show(pd.DataFrame(sorted(
    [v for v in agg.values() if v["num"] > 4],
    key=lambda entry: entry["net"],
    reverse=True,
)))

<ipython-input-16-e5b0eba07282>:14: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  entries = list(df_all.transpose().to_dict().values())


In [17]:
import pickle
from cytoolz import get as pick_from_dict
from cytoolz import sliding_window
import glob
import itertools
from blizzard import auction_summary
import re
import numpy as np
import datetime
np.set_printoptions(suppress=True)

_names = """
flask of the frost wyrm
flask of endless rage
flask of stoneblood
titanium bar
indestructible potion
eternal shadow
oily blackmouth
elixir of mighty thoughts
frostweave cloth
wildvine
felcloth
lesser healing potion
runic mana potion
greater planar essence
netherweave cloth
greater healing potion
crystallized water
purple lotus
bruiseweed
arcane dust
tiger lily
crystallized fire
major healing potion
potion of speed
potion of wild magic
lichbloom
pygmy oil
adder's tongue
imbued vial
icethorn
frost lotus
enchanted vial
goldclover
crystallized life
eternal life
deadnettle
elixir of accuracy
lesser planar essence
eternal fire
eternal water
chaotic skyflare diamond
skyflare diamond
potent monarch topaz
monarch topaz
runed scarlet ruby
scarlet ruby
purified twilight opal
twilight opal
potion of speed
potion of wild magic
lichbloom
frostweave cloth
infinite dust
greater cosmic essence
dream shard
eternal belt buckle
titansteel bar
titansteel spellblade
mageweave cloth
rough stone
copper bar
coarse stone
linen cloth
bronze bar
heavy stone
moss agate
wool cloth
medium leather
steel bar
solid stone
mithril bar
mageweave cloth
dense stone
thorium bar
runecloth
fel iron bar
mote of earth
mote of fire
netherweave cloth
adamantite bar
cobalt bar
frostweave cloth
crystallized water
crystallized earth
crystallized fire
borean leather
eternal shadow
saronite bar
""".strip().splitlines()
desired_names = list(set(_names))
desired_ids = [items.get_id(name) for name in desired_names if not name.startswith("#")]


date_re = re.compile(r"202.-..-..T..-..-..")
def _timestamp_from_name(name):
    cap = date_re.search(name).group(0)
    return datetime.datetime.strptime(cap, "%Y-%m-%dT%H-%M-%S").timestamp()


by_item = {}
timeseries = []

for snapshot in sorted(glob.glob("bliz-ah/*")):
    with open(snapshot, "rb") as f:
        entries = pickle.load(f).values()
        timeseries.extend(
            [
                {"timestamp": _timestamp_from_name(snapshot), "item_name": items.get_name(item_id), **auction_summary(item_data)}
                for (item_id, item_data) in zip(desired_ids, entries)
            ]
        )
        for (item_id, item_data) in zip(desired_ids, entries):
            name = items.get_name(item_id)
            by_item[name] = by_item.get(name, []) + [{"timestamp": date_re.search(snapshot).group(0), "item_name": items.get_name(item_id), **auction_summary(item_data)}]

def timeseries_of(key, default=np.nan):
    def _timeseries_of(item_name):
        return np.array([v.get(key, default) for v in by_item[item_name]])
    return _timeseries_of

def omit_outliers(series, sigma=2, window=5):
    mavg = [np.average(w) for w in sliding_window(window, series)]
    std = np.std(mavg)
    return np.array([x if np.abs(x) < sigma*std else np.nan for x in mavg])

def avg_finite(arr):
    return np.average(arr[np.isfinite(arr)])

In [18]:
import plotly.express as px
for yvalue in ["num", "min", "p50", "d_num", "d_min", "d_p50"]:
    df = pd.DataFrame(timeseries)
    df = df.sort_values(by="timestamp")

    if yvalue.startswith("d_"):
        dfg = df.pivot(index="timestamp", columns="item_name", values=yvalue[2:])
        dfg = dfg.diff().div(dfg.index.to_series().diff(), axis=0)
    else:
        dfg = df.pivot(index="timestamp", columns="item_name", values=yvalue)

    fig = px.scatter(dfg, title=yvalue, log_y=False)
    fig.write_html(f"{yvalue}.html")

In [19]:
def min_series(x):
    return omit_outliers(timeseries_of("avg_sell")(x), sigma=0.5, window=10)

wild_magic_pot = avg_finite(min_series("potion of wild magic") - (2*min_series("lichbloom") + min_series("pygmy oil") + np.array(1.8)))
speed_pot = avg_finite(min_series("potion of speed") - (2*min_series("adder's tongue") + min_series("pygmy oil") + np.array(1.8)))
wyrm = avg_finite(2*min_series("flask of the frost wyrm") - (5*min_series("icethorn") + 5*min_series("lichbloom") + min_series("frost lotus") + np.array(5)))
rage = avg_finite(2*min_series("flask of endless rage") - (5*min_series("goldclover") + 7*min_series("lichbloom") + min_series("frost lotus") + np.array(5)))
indestructible = avg_finite(2*min_series("indestructible potion") - (2*min_series("icethorn") + np.array(1.8)))

In [20]:
def item_infos(items):
    return pd.DataFrame([item_info(i) for i in items])

In [21]:
ii = item_infos(_names)
show(ii)

In [22]:
approaches("potion of speed")

{'market': {'profit': -1.6827,
  'recipe': ([potion of speed],
   2*[adder's tongue] + [imbued vial] + [pygmy oil]),
  'input_price_method': <function crafting.gold.<locals>._gold(item_inf)>,
  'output_price_method': <function crafting.gold.<locals>._gold(item_inf)>,
  'input_breakdown': {"[adder's tongue]": {'count': 2,
    'price': 0.9052,
    'total': 1.8104},
   '[imbued vial]': {'count': 1, 'price': 2.4963, 'total': 2.4963},
   '[pygmy oil]': {'count': 1, 'price': 1.3407, 'total': 1.3407},
   'TOTAL': 5.6474},
  'output_breakdown': {'[potion of speed]': {'count': 1,
    'price': 3.9647,
    'total': 3.9647},
   'TOTAL': 3.9647}},
 'flip': {'profit': 1.0545,
  'recipe': ([potion of speed],
   2*[adder's tongue] + [imbued vial] + [pygmy oil]),
  'input_price_method': <function crafting.gold.<locals>._gold(item_inf)>,
  'output_price_method': <function crafting.gold.<locals>._gold(item_inf)>,
  'input_breakdown': {"[adder's tongue]": {'count': 2,
    'price': 0.6928,
    'total': 1.3

In [23]:
item_info(item_name="spellweave")

{'id': 41595,
 'name': 'spellweave',
 'num_auctions': 16,
 'quantity': 16,
 'weight_sell': 198.18715,
 'avg_sell': 198.18715,
 'max': 199.9999,
 'p80': 197.9998,
 'p50': 197.9997,
 'p20': 197.9995,
 'wp80': 197.9998,
 'wp50': 197.9997,
 'wp20': 197.9995,
 'min': 197.9993,
 'realm_market_value': 154.8928,
 'realm_historical': 170.9999,
 'region_historical': 109.9998,
 'region_avg_sale_price': 79.8551,
 'sale_pct': 58.0,
 'sold_per_day': 189,
 'headroom': 309,
 'market_skew_pct': -9.42,
 'auction_skew_pct': -99.99}

In [24]:
item_info(item_name="bolt of imbued frostweave")

{'id': 41511,
 'name': 'bolt of imbued frostweave',
 'num_auctions': 30,
 'quantity': 49,
 'weight_sell': 131.74813469387757,
 'avg_sell': 24.556553333333333,
 'max': 300.998,
 'p80': 15.0499,
 'p50': 15.0499,
 'p20': 15.0,
 'wp80': 300.998,
 'wp50': 15.0499,
 'wp20': 15.0,
 'min': 15.0,
 'realm_market_value': 16.407,
 'realm_historical': 15.254,
 'region_historical': 12.4553,
 'region_avg_sale_price': 7.8691,
 'sale_pct': 45.3,
 'sold_per_day': 367,
 'headroom': 761,
 'market_skew_pct': 7.56,
 'auction_skew_pct': -99.99}

In [25]:
item_info(item_name="frozen orb")

{'id': 43102,
 'name': 'frozen orb',
 'num_auctions': 24,
 'quantity': 34,
 'weight_sell': 231.64608235294116,
 'avg_sell': 100.6653,
 'max': 700.0,
 'p80': 70.9899,
 'p50': 64.9998,
 'p20': 62.0,
 'wp80': 700.0,
 'wp50': 70.9899,
 'wp20': 63.0,
 'min': 60.0,
 'realm_market_value': 70.9495,
 'realm_historical': 70.9997,
 'region_historical': 67.0017,
 'region_avg_sale_price': 58.6907,
 'sale_pct': 84.2,
 'sold_per_day': 514,
 'headroom': 576,
 'market_skew_pct': -0.07,
 'auction_skew_pct': -99.99}